In [13]:
# Import Libraries

import numpy as np
import pandas as pd
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
from keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn import metrics
from sklearn.ensemble import RandomForestClassifier
from pyrsm import gains, gains_plot, lift, lift_plot, confusion, profit_max, ROME_max

In [14]:
# Read in data and split into training and test

data = pd.read_csv('../intuit75k_new.csv')
categorical_columns=['zip_bins','sex','zip801','zip804']
othercol=['numords','last','dollars','sincepurch','bizflag','owntaxprod','version1','upgraded','training']
keep=categorical_columns+othercol+['label']
combind_data=data.loc[:,keep]
combind_data[categorical_columns] = combind_data[categorical_columns].apply(lambda x: LabelEncoder().fit_transform(x))
X_train=combind_data.loc[combind_data.training==1].drop(columns='label').drop(columns='training')
y_train=combind_data.loc[combind_data.training==1].label
X_test=combind_data.loc[combind_data.training==0].drop(columns='label').drop(columns='training')
y_test=combind_data.loc[combind_data.training==0].label

In [22]:
# Create the parameter grid: rf_param_grid
rf_param_grid = {
    'n_estimators': range(140,160,1), 
    'max_depth': range(8,12)
}

# Instantiate the regressor: clf
clf = RandomForestClassifier()

# Perform grid search
grid_clf = GridSearchCV(
    param_grid=rf_param_grid, estimator=clf, scoring="roc_auc",cv=5,verbose=1
)

# Fit grid_mse to the data
grid_clf.fit(X_train,y_train)


Fitting 5 folds for each of 80 candidates, totalling 400 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 400 out of 400 | elapsed: 31.3min finished


GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False,
                                              rando

In [23]:
# Find best parameters and highest AUC

print("Best parameters found: ", randomized_clf.best_params_)
print("higest auc found: ", np.abs(randomized_clf.best_score_))

Best parameters found:  {'max_depth': 8, 'n_estimators': 200}
higest auc found:  0.7696539151484771


In [24]:
# Get AUC

preds =randomized_clf.predict_proba(X_test)
fpr, tpr, thresholds = metrics.roc_curve(y_test.values, preds[:, 1])
auc_rf = metrics.auc(fpr, tpr)
auc_rf

0.7644931286704387

In [25]:
#scaled profit

margin=60
cost=1.41
breakeven_rate=cost/margin

total=763334
testdata=225000
testdata2=combind_data.loc[combind_data.training==0]
testdata2['randomforest']=preds[:,1]
testdata2['pred_click']=1
testdata2.loc[testdata2['randomforest']/2<breakeven_rate,'pred_click']=0
tp=testdata2.loc[testdata2.pred_click==1].loc[testdata2.label==1]
pred_true=sum(testdata2.pred_click)
pred_true_rate=pred_true/len(testdata2)
send_number=total*pred_true_rate
adj_response_rate=len(tp)/pred_true/2
exp_buyers=adj_response_rate*send_number
totalcost=send_number*cost
exp_profit=exp_buyers*margin-totalcost

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.6/dist-packages/pandas/core/indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_index

In [26]:
exp_profit

443818.67205866677